<img style="float: left;;" src='Figures/alinco.png' /></a>

# <center> <font color= #000047> Módulo 1: Crear una Red Neuronal Recurrente en TensorFlow 2.0



## Paso 1: Importar las librerías necesarias

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb

In [2]:
tf.__version__

'2.2.0'

## Paso 2: Pre procesado de datos


### Configurar parámetros del dataset

In [3]:
number_of_words=20000
max_len = 100

### Carga del dataset de IMDB

In [4]:
(x_train,y_train),(x_test, y_test) = imdb.load_data(num_words=number_of_words)

In [5]:
y_train

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

### Cortar secuencias de texto de la misma longitud

In [6]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_len)

In [7]:
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_len)

### Configurar parámetros de la capa de Embedding

In [8]:
vocab_size=number_of_words
embed_size=128

## Paso 3: Construir la Red Neuronal Recurrente

### Definir el modelo

In [9]:
model=tf.keras.Sequential()

### Añadir la capa de embedding

In [10]:
model.add(tf.keras.layers.Embedding(vocab_size,embed_size, input_shape=(x_train.shape[1],)))

### Añadir la capa de LSTM

- unidades: 128
- función de activación: tanh

In [ ]:
#!pip install -U numpy==1.18.5 --user

In [11]:
model.add(tf.keras.layers.LSTM(units=128, activation='tanh'))
#model.add(Dropout(0.2))

### Añadir la capa totalmente conectada de salida

- unidades: 1
- función de activación: sigmoid

In [12]:
model.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 128)          2560000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


### Compilar el modelo

In [14]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

### Entrenar el modelo

In [ ]:
model.fit(x_train,y_train, epochs=10, batch_size=128)

Epoch 1/10
196/196 [==============================] - 24s 120ms/step - loss: 0.4903 - accuracy: 0.7688
Epoch 2/10
196/196 [==============================] - 25s 128ms/step - loss: 0.2923 - accuracy: 0.8834
Epoch 3/10
196/196 [==============================] - 28s 143ms/step - loss: 0.2312 - accuracy: 0.9123
Epoch 4/10
196/196 [==============================] - 38s 191ms/step - loss: 0.1906 - accuracy: 0.9292
Epoch 5/10
196/196 [==============================] - 35s 178ms/step - loss: 0.1555 - accuracy: 0.9426
Epoch 6/10
 26/196 [==>...........................] - ETA: 28s - loss: 0.1120 - accuracy: 0.9606

### Evaluar el modelo

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test,y_test)
print(f'Test accuracy : {test_accuracy}')

In [ ]:
#mejorar el accuracy de la red 1

### Otro Ejemplo: Predecir el precio de las acciones de Google

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
dataset_train = pd.read_csv('Data/Google_Stock_Price_Train.csv')


In [ ]:
dataset_train.info()

In [ ]:
dataset_train['Date'].min(),dataset_train['Date'].max()

In [ ]:
training_set = dataset_train.iloc[:,1:2].values


In [ ]:
training_set.shape[0]

In [ ]:
# Preprocesado de los datos
from sklearn.preprocessing import MinMaxScaler

In [ ]:
sc=MinMaxScaler()
training_set_scaled = sc.fit_transform(training_set)

In [ ]:
#visualizar los datos
plt.figure(figsize=(6,7))
plt.plot(training_set_scaled, color='r', label='Google Stock')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
# Crear una estructura de datos que me permita añadir a la entrada de la RNN

# crear un conjunto de datos de dimension 60, 
X_train=[]
y_train=[]

for i in range(60,training_set.shape[0]):
    X_train.append(training_set_scaled[i-60:i,0])
    y_train.append(training_set_scaled[i,0])

X_train, y_train = np.array(X_train),np.array(y_train)

# reshape convertir la matriz a un tensor de 3D
(batch_size,timesteps) = X_train.shape

#tensor 3D
X_train = np.reshape(X_train, (batch_size,timesteps,1))



In [ ]:
# Construyendo la red

model_stock = tf.keras.Sequential()
model_stock.add(tf.keras.layers.LSTM(units=50,return_sequences=True, input_shape=(timesteps,1)))
model_stock.add(tf.keras.layers.Dropout(0.2))
model_stock.add(tf.keras.layers.LSTM(units=50,return_sequences=True))
model_stock.add(tf.keras.layers.Dropout(0.2))
model_stock.add(tf.keras.layers.LSTM(units=50))
model_stock.add(tf.keras.layers.Dropout(0.2))
model_stock.add(tf.keras.layers.Dense(units=1))



In [ ]:
model_stock.summary()

In [ ]:
#Compilar el modelo
model_stock.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
#Entrenar al modelo
model_stock.fit(X_train, y_train, epochs=100, batch_size=32)

In [ ]:
model_stock.save('model_stock.hd5')

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
dataset_test = pd.read_csv('Data/Google_Stock_Price_Test.csv')

In [ ]:
real_stock_prices = dataset_test.iloc[:,1:2].values

In [ ]:
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis=0)
dataset_total.shape

In [ ]:
dataset_test.shape

In [ ]:
first_day_index = len(dataset_total) - len(dataset_test)
inputs = dataset_total[first_day_index - 60:].values
inputs=inputs.reshape(-1,1)

#escalar los datos
inputs = sc.transform(inputs)

In [ ]:
X_test=[]

for i in range(60,80):
    X_test.append(inputs[i-60:i,0])

X_test = np.array(X_test)

# reshape convertir la matriz a un tensor de 3D
(batch_size,timesteps) = X_test.shape

#tensor 3D
X_test = np.reshape(X_test, (batch_size,timesteps,1))


In [ ]:
# predicción 
predicted_stock_price = model_stock.predict(X_test)
# invertir el escalamiento
predicted_stock_price = sc.inverse_transform(predicted_stock_price)


In [ ]:
#visualizar los datos con la predicción
plt.figure(figsize=(6,7))
plt.plot(real_stock_prices, color='r', label='Real Google Stock')
plt.plot(predicted_stock_price, color='g', label='Predicted Google Stock')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

## Generación de texto usando RNN

Cuando se trabaja con texto en un modelo de lenguaje natural (recordar que los tokens son palabras o caracteres), usando cualquier red podríamos modelar la probabilidad del siguiente token (palabra) para construir un modelo de generación de texto. 

Este modelo de lenguaje natural podría capturar la estructura estadística del texto completo. Entonces, podríamos entrenar una red neuronal para predecir el siguiente carácter, ó de manera similar, podríamos entrenar al modelo para que prediga la siguiente palabra, dada una secuencia de palabras. En este apartado implementaremos un modelo a nivel de caracter.


<img src="Figures/63.png" alt="Grayscale Image" width="600">

### Implementing in Tensorflow
#### The Dataset
Usaremos un conjunto de datos que contiene las obras de Shakespeare.

In [ ]:
import tensorflow as tf
import numpy as np

path = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [ ]:
text = open(path,'r').read()


In [ ]:
print(text[:200])

### Data Pre-Processing

Antes del entrenamiento, necesitamos asignar cadenas a números, extraer secuencias parcialmente superpuestas y empaquetarlas en una matriz numpy 3D de forma (secuencias, maxlen, caracteres únicos). 

In [ ]:
def process_text(file_path):
    text = open(file_path, 'rb').read().decode(encoding='utf-8')
    vocab = sorted(set(text))
    
    # crear un mapeo de los caracteres único con indices enteros
    char2indx = {u: i for i,u in enumerate(vocab)}
    indx2char = np.array(vocab)
    text_as_int = np.array([char2indx[c] for c in text])
    
    return text_as_int, vocab, char2indx, indx2char

In [ ]:
def split_input_target(chunk):
    input_text, target_text = chunk[:-1], chunk[1:]
    return input_text, target_text


In [ ]:
def create_dataset(text_as_int, seq_length=100, batch_size=64, buffer_size=10000):
    char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
    dataset = char_dataset.batch(seq_length+1, drop_remainder=True).map(split_input_target)
    dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)
    return dataset

### Construyendo la RNN

La red que construiremos será una red con una sola capa LSTM seguida de una ,capa densa con una función de activación softmax.

In [ ]:
def build_model(vocab_size, embedding_dim=256, rnn_units=1024, batch_size =64):
    model=tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(vocab_size)
    ])
    
    return model

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
text_as_int, vocab, char2indx, indx2char = process_text(path)

In [ ]:
text_as_int

In [ ]:
vocab

In [ ]:
char2indx

In [ ]:
indx2char

In [ ]:
dataset = create_dataset(text_as_int)

In [ ]:
dataset

In [ ]:
model = build_model(vocab_size=len(vocab))
model.summary()

### Compile and Train the model

Usaremos la función de pérdida `categorical_crossentropy` para entrenar el modelo.

In [ ]:
model.compile(optimizer='adam', loss=loss)


### Train the model

In [ ]:
history = model.fit(dataset, epochs=1)


In [ ]:
model.save_weights('gen_text.h5', save_format='h5')
model = build_model(vocab_size=len(vocab), batch_size=1)
model.load_weights('gen_text.h5')
model.summary()

## Generate text

In [ ]:
def generate_text(model, char2indx, indx2char, start_string, generate_char_num=1000):
    input_eval = [char2indx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated =[]
    model.reset_states()
    
    for i in range(generate_char_num):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions,0)
        
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], axis=0)
        text_generated.append(indx2char[predicted_id])
    
    return start_string + ''.join(text_generated)
        
    

In [ ]:
user_string = input("Escriba el texto inicial: ")
generated_text = generate_text(model, char2indx, indx2char, start_string=user_string, generate_char_num=1000)

In [ ]:
print(generated_text)